In [ ]:
# SUDA2 ALGORITHM IMPLEMENTATION

# This script computes SUDA (Special Uniques Detection Algorithm) scores on stratified samples
# to measure privacy disclosure risk in census microdata

import pandas as pd
import itertools
from multiprocessing import Pool, cpu_count
from collections import defaultdict
import math

# Global variable for maximum MSU (Minimal Sample Unique) size
MAX_MSU_SIZE = None

def normalization_constant(k):
    """Calculate normalization constant for k variables"""
    return sum(math.comb(k, r) * (2 ** (k - r)) for r in range(1, k + 1))

def detect_msus(df, key_vars, max_msu_size):
    """
    Core SUDA algorithm: detects Minimal Sample Uniques (MSUs)

    MSUs are the smallest combinations of variables that uniquely identify individuals.
    For each record, we find all minimal combinations of quasi-identifiers that
    make that record unique in the sample.
    """
    k = len(key_vars)  # Total number of quasi-identifier variables
    msu_results = []   # Store all detected MSUs
    variable_contributions = defaultdict(int)  # Track how much each variable contributes to risk

    # Initialize record-level scoring structure
    record_scores = defaultdict(lambda: {
        "suda_score": 0,           # Total SUDA score for this record
        "msu_counts": defaultdict(int),  # Count of MSUs by size
        "msus": [],                # List of all MSUs for this record
        "variables": defaultdict(int)    # Variable-specific contributions
    })

    # Search through all possible combinations of variables, from size 1 to max_msu_size
    for r in range(1, min(k, max_msu_size) + 1):
        # Generate all combinations of r variables from the key variables
        for cols in itertools.combinations(key_vars, r):
            # Group records by current variable combination and count occurrences
            grouped = df.groupby(list(cols)).size()
            # Find combinations that appear exactly once (unique combinations)
            unique_combos = grouped[grouped == 1].index

            # Handle the case where we have only one grouping variable
            if not isinstance(unique_combos, pd.MultiIndex):
                unique_combos = [(val,) for val in unique_combos]
            else:
                unique_combos = list(unique_combos)

            # Process each unique combination
            for uc in unique_combos:
                # Find the specific record that has this unique combination
                mask = (df[list(cols)] == pd.Series(uc, index=cols)).all(axis=1)
                matching_rows = df[mask]
                # Skip if somehow more than one record matches (shouldn't happen)
                if matching_rows.shape[0] != 1:
                    continue

                record_idx = matching_rows.index[0]
                uc_dict = dict(zip(cols, uc))

                # Check if this combination is minimal (no subset also makes record unique)
                is_minimal = True
                if len(cols) > 1:  # Only check for minimality if combination has >1 variable
                    # Test all smaller subsets of current combination
                    for sub_r in range(1, len(cols)):
                        for sub_cols in itertools.combinations(cols, sub_r):
                            sub_uc = tuple(uc_dict[c] for c in sub_cols)
                            sub_mask = (df[list(sub_cols)] == pd.Series(sub_uc, index=sub_cols)).all(axis=1)
                            # If any subset is also unique, current combination is not minimal
                            if df[sub_mask].shape[0] == 1:
                                is_minimal = False
                                break
                        if not is_minimal:
                            break

                # If combination is minimal, add to MSU results and calculate scores
                if is_minimal:
                    # Weight decreases as combination size increases (smaller = riskier)
                    weight = 2 ** (k - r)

                    # Store MSU information
                    msu_results.append({
                        'record_index': int(record_idx),
                        'msu_combination': list(cols)
                    })

                    # Update record-level scores
                    record_scores[int(record_idx)]["suda_score"] += weight
                    record_scores[int(record_idx)]["msu_counts"][r] += 1
                    record_scores[int(record_idx)]["msus"].append(list(cols))

                    # Track variable-level contributions
                    for v in cols:
                        variable_contributions[v] += weight
                        record_scores[int(record_idx)]["variables"][v] += weight

    # Convert nested defaultdicts to regular dicts for JSON serialization
    record_scores_serializable = {
        int(rec_id): {
            "suda_score": data["suda_score"],
            "msu_counts": dict(data["msu_counts"]),
            "msus": data["msus"],
            "variables": dict(data["variables"]),
        } for rec_id, data in record_scores.items()
    }
    var_contrib_serializable = dict(variable_contributions)
    return msu_results, record_scores_serializable, var_contrib_serializable

def parallel_suda(df, key_vars, max_msu_size):
    """
    Wrapper function for parallel processing of SUDA algorithm
    Currently uses single process but structure allows for easy parallelization
    """
    # Use all available CPU cores minus one
    num_cores = max(1, cpu_count() - 1)
    pool = Pool(num_cores)

    # Create work chunks (currently just one chunk containing full dataset)
    chunks = [(df, key_vars, max_msu_size)]
    results = pool.starmap(detect_msus, chunks)
    pool.close()
    pool.join()

    # Combine results from all worker processes
    all_msus = []
    combined_scores = defaultdict(lambda: {
        "suda_score": 0,
        "msu_counts": defaultdict(int),
        "msus": [],
        "variables": defaultdict(int)
    })
    var_contrib = defaultdict(int)

    # Merge results from all chunks
    for msus, record_scores, var_scores in results:
        all_msus.extend(msus)
        for rec_id, val in record_scores.items():
            combined_scores[rec_id]["suda_score"] += val["suda_score"]
            for k, v in val["msu_counts"].items():
                combined_scores[rec_id]["msu_counts"][k] += v
            combined_scores[rec_id]["msus"].extend(val["msus"])
            for var, v in val["variables"].items():
                combined_scores[rec_id]["variables"][var] += v
        for var, v in var_scores.items():
            var_contrib[var] += v

    return all_msus, combined_scores, var_contrib

def generate_output(df, key_vars, msus, record_scores, var_contrib, max_msu_size,
                    output_prefix="suda_output"):
    """
    Generate SUDA output files in standard format

    Creates:
    - Summary file with dataset statistics
    - Records CSV with individual SUDA scores and metrics
    - Attribute contribution file showing variable importance
    """
    k = len(key_vars)
    n_records = len(df)
    normalization_const = normalization_constant(k)
    sample_unique_count = len(record_scores)
    num_msus = len(msus)
    total_score = sum(r["suda_score"] for r in record_scores.values())

    # Create summary statistics file
    with open(f"{output_prefix}_summary.txt", "w") as f:
        f.write("+++++++++++++++++++++++++++++++++++++++++++++++++++\n")
        f.write("Running SUDA with debug level=0\n")
        f.write("file:___suda-processedData_.txt\n")
        f.write("IDs:1\n")
        f.write("detailed output:1\n")
        f.write("samp. fract:0.010000\n")
        f.write(f"set={n_records} att={k}\n")
        f.write(f"max_range={df.shape[1]} max_records={n_records}\n")
        f.write(":::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::\n")
        f.write("<<<<< Loading data - may take a few minutes >>>>>\n")
        f.write("Attribute details:\n")
        # Document each variable's range for reproducibility
        for idx, col in enumerate(key_vars, start=1):
            f.write(f"{idx} col#{idx+1} att '{col}' min value {df[col].min()} max value {df[col].max()}\n")
        f.write(f"Search for minimal sample uniques up to size {max_msu_size} from {k} attributes\n")
        f.write(f"***The number of records read in is {n_records}***\n")
        f.write("Applying SUDA...\n")
        f.write(":::::DIS_SUDA DETAILS::::::::::::::::::::::::::::::::::::::::::::\n")
        f.write(f"Sample Unique records {sample_unique_count}\n")
        f.write(f"Records in pairs {num_msus}\n")
        f.write(f"P(cm|um) = {num_msus / n_records:.6f}\n")
        f.write("+++++++++++++++++++++++++++++++++++++++++++++++++++\n")

    # Generate detailed record-level output
    record_data = []
    for idx, data in record_scores.items():
        suda_score = data["suda_score"]

        # Calculate proportion of lattice (normalized SUDA score)
        proportion_lattice = suda_score / math.factorial(k) if k > 0 else 0

        # Count MSUs by their size (1-variable, 2-variable, etc.)
        msu_sizes = {
            f"MSU_size_{i}": data["msu_counts"].get(i, 0)
            for i in range(1, max_msu_size + 1)
        }

        # Calculate percentage contribution of each variable to this record's risk
        var_contribs = {
            f"{v}_contribution_%": round(w / suda_score * 100, 2) if suda_score > 0 else 0
            for v, w in data["variables"].items()
        }

        # Compile all metrics for this record
        row = {
            "ID": idx,
            "suda_score": suda_score,
            "proportion_of_lattice": round(proportion_lattice, 6)
        }
        row.update(msu_sizes)
        row.update({f"{var}_contribution_%": var_contribs.get(f"{var}_contribution_%", 0) for var in key_vars})

        record_data.append(row)

    # Save record-level results to CSV
    pd.DataFrame(record_data).to_csv(f"{output_prefix}_records.csv", index=False)

    # Generate variable-level contribution summary
    with open(f"{output_prefix}_attribute_contrib.txt", "w") as f:
        for var in key_vars:
            # Calculate what percentage of total risk this variable contributes
            percent = var_contrib[var] / total_score * 100 if total_score > 0 else 0
            f.write(f"col#{key_vars.index(var)+2} att '{var}' percentage contribution {percent:.4f}\n")



In [ ]:
# EXECUTING ALGORITHM ON SAMPLES

# Load stratified sample 2 data
data = pd.read_csv("s1_ns2011.csv")

# Run SUDA algorithm on all samples
if __name__ == "__main__":
    # Load the sample dataset
    df = data

    # Define quasi-identifier variables (must match dataset column names)
    # These are demographic variables that could potentially re-identify individuals
    key_vars = ["persons", "hhwt", "gq", "regionw", "ownershipd"]

    # Set maximum MSU size to consider all possible variable combinations
    MAX_MSU_SIZE = len(key_vars)

    # Execute SUDA analysis using parallel processing
    msus, record_scores, var_contrib = parallel_suda(df, key_vars, MAX_MSU_SIZE)

    # Generate comprehensive output files with results
    generate_output(df, key_vars, msus, record_scores, var_contrib, MAX_MSU_SIZE, output_prefix="s1_ns2011")

In [ ]:
# TOTAL SUDA SCORE AND ERROR %

# Load SUDA results from the generated CSV file
df = pd.read_csv("s1_ns2011_records.csv")

# Calculate total SUDA score by summing individual record scores
suda_total = df["suda_score"].sum()
print(suda_total)

# Compare sample SUDA score against true population risk
# Population risk of 15061 was calculated from the full census dataset
population_risk = 15061

# Calculate percentage error between sample estimate and true population value
# Positive error = sample overestimates risk, Negative error = sample underestimates risk
percent_error = ((suda_total - population_risk) / population_risk) * 100
print(f"% Error: {percent_error:.2f}%")

29872
% Error: 98.34%
